# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117113e+02     1.568772e+00
 * time: 0.11998581886291504
     1     1.040227e+01     9.394079e-01
 * time: 0.9766418933868408
     2    -1.172722e+01     9.370649e-01
 * time: 1.111163854598999
     3    -3.379371e+01     7.557226e-01
 * time: 1.2709848880767822
     4    -4.715713e+01     6.297985e-01
 * time: 1.419252872467041
     5    -5.664697e+01     2.129930e-01
 * time: 1.5645339488983154
     6    -5.951903e+01     1.917551e-01
 * time: 1.6780939102172852
     7    -6.072375e+01     7.556523e-02
 * time: 1.7919340133666992
     8    -6.127273e+01     4.169593e-02
 * time: 1.8993189334869385
     9    -6.160965e+01     3.631704e-02
 * time: 2.014737844467163
    10    -6.180557e+01     2.880499e-02
 * time: 2.128154993057251
    11    -6.192975e+01     2.857848e-02
 * time: 2.234128952026367
    12    -6.201948e+01     1.915892e-02
 * time: 2.3478848934173584
    13    -6.208648e+01     1.374419e-02
 * time: 2.46068096160888

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671059
    AtomicLocal         -18.8557654
    AtomicNonlocal      14.8522637
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485360 
    Xc                  -19.3336815

    total               -62.261666460744
